In [1]:
import pandas as pd
import numpy as np

In [2]:
from pandas.core.window.common import _flex_binary_moment
from pandas.core.window.expanding import _Rolling_and_Expanding

def weighted_average(self, weights, **kwargs):
    weights = self._shallow_copy(weights)
    window = self._get_window(weights)

    def _get_weighted_average(X, Y):
        X = X.astype('float64')
        Y = Y.astype('float64')
        sum_f = lambda x: x.rolling(window, self.min_periods, center=self.center).sum(**kwargs)
        return sum_f(X * Y) / sum_f(Y)

    return _flex_binary_moment(self._selected_obj, weights._selected_obj,
                               _get_weighted_average, pairwise=True)

_Rolling_and_Expanding.weighted_average = weighted_average

In [ ]:
df = pd.DataFrame(
    {'myvalues': [1,2,3,4,5],
    'myweights': [5,4,3,2,1]
    }
)

df['wavg'] = df['myvalues'].expanding().weighted_average(weights=df['myweights'])
df['moving_avg'] = df['myvalues'].expanding().mean()
df

In [3]:
sensors = pd.read_csv('../data/01_raw/metadata/wind_turbine_data.csv', sep=';',  header = 0, names= ['id', 'power', 'commissioning_date', 'hub_height', 'diameter', 'nuts_id', 'lon', 'lat', 'NUTS1'], usecols= ['power', 'commissioning_date', 'hub_height', 'diameter', 'nuts_id', 'lon', 'lat'], parse_dates= ['commissioning_date'], infer_datetime_format= True)

display(sensors.head(3)); display(sensors.shape)

,power,commissioning_date,hub_height,diameter,nuts_id,lon,lat
0,1500.0,2002-01-12,61.5,77.0,DE145,9.628233,48.532278
1,1500.0,2002-01-12,61.5,77.0,DE145,9.636193,48.532907
2,750.0,1999-07-07,70.0,48.0,DE145,9.630243,48.391163


(24861, 7)

In [4]:
sensors_datesorted = sensors.sort_values('commissioning_date')

sensors_datesorted.loc[(sensors['nuts_id']=='DEF0C'), ['commissioning_date', 'power', 'lat', 'lon']].tail(3)

,commissioning_date,power,lat,lon
23455,2016-12-15,3400.0,54.720280,9.301851
23293,2017-03-13,3050.0,54.389937,9.450079
23294,2017-03-27,3050.0,54.388749,9.455410


In [5]:
def _power_weighted_average(coord):
     return np.average(coord, weights=sensors_datesorted.loc[coord.index, 'power'])

sensors_daily_aggregated = sensors_datesorted \
    .groupby(by=['nuts_id', 'commissioning_date'], sort=False) \
        .agg(
            power = ('power', 'sum'),
            lat = ('lat', _power_weighted_average),
            lon = ('lon', _power_weighted_average),
            )

In [ ]:
sensors_daily_aggregated.tail(3)

In [ ]:
sensors_daily_aggregated.loc[('DEF0C',), :].tail(3)

In [ ]:
power_centroids_mts = sensors_daily_aggregated \
    .groupby(level='nuts_id') \
        .expanding() \
            .weighted_average(weights=sensors_daily_aggregated['power']) 

power_centroids_mts.loc[('DEF0C',),:].tail(3)

In [ ]:
power_centroids_mts = sensors_daily_aggregated \
    .groupby(level='nuts_id') \
        .expanding() \
            .mean() 

power_centroids_mts.loc[('DEF0C',),:].tail(3) 

In [ ]:
sensors_daily_aggregated['power']

In [ ]:
def weighted_avg(df: pd.DataFrame, vals: str, weights: str) -> pd.Series:
    window = pd.DataFrame({'vals':[], 'weights': []})
    wavg = []
    for v, w in zip(df[vals], df[weights]):
        window = window.append({'vals': v, 'weights': w}, ignore_index=True)
        wavg.append( window['vals'] * window['weights'] / window['weights'].sum() )
    return wavg

def weighted_avg2(mydf: pd.DataFrame) -> pd.Series:
    window = pd.DataFrame({'vals':[], 'weights': []})
    wavg = []
    for v, w in zip(mydf['myvalues'], mydf['myweights']):
        
    return wavg 

df['wavg_simple'] = df.apply(func=weighted_avg, args=('myvalues', 'myweights'))
df

In [ ]:
def wavg():
    for nuts_id in :

df['wavg'] = df['myvalues'].expanding().weighted_average(weights=df['myweights'])
df['moving_avg'] = df['myvalues'].expanding().mean()

In [ ]:
sensors_daily_aggregated.loc[('DEF0C',), :].tail(10)

In [ ]:
def weighted_average(df, data_col, weight_col, by_col, by_col2):
    df['_data_times_weight'] = df[data_col] * df[weight_col]
    df['_weight_where_notnull'] = df[weight_col] * pd.notnull(df[data_col])
    g = df.groupby([by_col, by_col2], sort=False)
    result = g['_data_times_weight'].sum() / g['_weight_where_notnull'].sum()
    del df['_data_times_weight'], df['_weight_where_notnull']
    return result

pwr_centroids_lat = weighted_average(df=sensors_daily_aggregated, by_col='nuts_id', by_col2='commissioning_date', data_col='lat', weight_col='power')
pwr_centroids_lon = weighted_average(df=sensors_daily_aggregated, by_col='nuts_id', by_col2='commissioning_date', data_col='lon', weight_col='power')


In [ ]:
power_centroids_mts = pd.concat([pwr_centroids_lat, pwr_centroids_lon], axis=1)
power_centroids_mts.columns = ['lat', 'lon']
power_centroids_mts.loc[('DEF0C',), :].tail(10)

In [ ]:
# get_wavg = lambda g: np.average(g['data'], weights = g['weights'])
get_wavg = lambda g: np.average(g['lat'], weights = g['power'])

grouped = sensors_daily_aggregated.groupby(['nuts_id'])

In [ ]:
grouped

In [ ]:
power_centroids_mts_A = grouped.expanding().apply(get_wavg)

In [ ]:
power_centroids_mts_A.loc[('DEF0C',)].tail(3)

In [ ]:
power_centroids_mts.loc['DEF0C'].tail(3)

In [ ]:
window = pd.DataFrame( {'vals':[1,2,3], 'weights': [4,5,6]})
window = window.append({'vals': 4, 'weights': 7}, ignore_index=True)
window['vals'] * window['weights']

In [ ]:
s = pd.Series([1, 2, 3, 4, 5])

display( s )
display( s.cumsum() )
display( s.expanding().sum() )
display( s.expanding().agg(func=sum) )
display( s.expanding().apply(func=np.sum) )

In [ ]:
import weightedcalcs as wc
calc = wc.Calculator("power") 

In [ ]:
sensors_daily_aggregated.columns

In [ ]:
grouped = sensors_daily_aggregated.groupby(['nuts_id'])

In [ ]:
expmean = grouped.expanding()['lat'].mean()

expmean.tail(3)

In [ ]:
calc.mean(grouped.expanding(), 'lat')

In [ ]:
sensors_daily_aggregated.loc[('DEF0C',), :].tail(3)

In [ ]:
sensors_datesorted.loc[sensors_datesorted.nuts_id=='DEF0C', ['commissioning_date', 'power', 'lat', 'lon']].tail()

In [ ]:
df = sensors_datesorted.groupby('nuts_id').expanding().apply(lambda x: x['lat'] * x['power'] / x['power'].sum() )

In [ ]:
static_lat = sensors_datesorted.groupby('nuts_id').apply(lambda x: (x['lat'] * x['power']).sum() / x['power'].sum() )


In [ ]:
dynamic_lat = sensors_datesorted.groupby('nuts_id').expanding().apply(lambda x: (x['lat'] * x['power']).sum() / x['power'].sum() )

In [ ]:
sensors_datesorted.tail()

In [9]:
single_district_data = sensors_daily_aggregated_simpleindex.loc[sensors_daily_aggregated_simpleindex['nuts_id'] == 'DEF0C'].set_index('commissioning_date')
single_district_data.tail(3)

,nuts_id,power,lat,lon
commissioning_date,,,,
2016-12-15,DEF0C,3400.0,54.720280,9.301851
2017-03-13,DEF0C,3050.0,54.389937,9.450079
2017-03-27,DEF0C,3050.0,54.388749,9.455410


In [ ]:
single_district_data[['lat', 'lon']].mean()

In [ ]:
single_district_data.expanding().apply(np.mean)

In [ ]:
single_district_data.expanding().apply( lambda x: np.mean(x) )

In [ ]:
single_district_data.rolling(window=len(single_district_data), min_periods=1, axis=0).apply( lambda x: x.sum() , raw=False)

In [ ]:
single_district_data.rolling(window=len(single_district_data), min_periods=1, axis=0).apply( func=np.average, kwargs={'axis': 0, 'weights': single_district_data['power']} )

In [7]:
sensors_daily_aggregated_simpleindex = sensors_daily_aggregated.reset_index()

sensors_daily_aggregated_simpleindex.tail()

,nuts_id,commissioning_date,power,lat,lon
13586,DEE0D,2017-11-01,3000.0,52.840646,11.596993
13587,DE724,2017-11-01,2400.0,50.884330,8.491895
13588,DE11D,2017-11-01,3000.0,49.034109,10.177751
13589,DEB15,2017-11-03,2400.0,49.639484,7.206280
13590,DEB3H,2017-12-01,3300.0,49.170525,8.239301


In [13]:
single_district_data[['lat', 'lon']].tail()

,lat,lon
commissioning_date,,
2016-09-28,54.522999,9.402982
2016-12-13,54.385309,9.464164
2016-12-15,54.720280,9.301851
2017-03-13,54.389937,9.450079
2017-03-27,54.388749,9.455410


In [12]:
power_centroids_ts = single_district_data[['lat', 'lon']] \
.expanding() \
.weighted_average(weights=single_district_data['power']) 

power_centroids_ts.tail()

,lat,lon
commissioning_date,,
2016-09-28,54.671404,9.306583
2016-12-13,54.670255,9.307216
2016-12-15,54.670455,9.307194
2017-03-13,54.669452,9.307705
2017-03-27,54.668452,9.308232
